# XGBoost_in_R
----------

* This script makes use of *earlier* versions of the datasets later uploaded to Kaggle.
* This script was originally written in RStudio, but was ported to Jupyter.

#### It should serve as a guide as to what kind of results 'might' be seen if similar strategies are followed.

----------

Apart from the initial prepwork, this script consists of 2 shots:

* Shot-1:
    * XGBoost is run with *minimal* hyper tuning.
* Shot-2:
    * XGBoost is run with *some* hyper tuning.

**Results**: Neither shots were able to correctly classify even a single exoplanet :-(

----------

Points to note:

* The data used in both shots was unnormalised, better results might be seen if the data was first normalised.

![Nu Orionis](http://www.danlessmann.com/FullSizeImages/Astro/M43-2006-12-22_Crop_Sharpen_filtered.jpg)

In [1]:
library(readr)
install.packages("xgboost")
library(xgboost)
install.packages("caret")
library(caret)
install.packages("e1071")
library(e1071)
install.packages("ggfortify")
library(ggfortify)


The downloaded binary packages are in
	/var/folders/q5/3j5mlj012vv_ys8bj7mz3vb00000gn/T//RtmpHRAmUq/downloaded_packages

The downloaded binary packages are in
	/var/folders/q5/3j5mlj012vv_ys8bj7mz3vb00000gn/T//RtmpHRAmUq/downloaded_packages


Loading required package: lattice
Loading required package: ggplot2



The downloaded binary packages are in
	/var/folders/q5/3j5mlj012vv_ys8bj7mz3vb00000gn/T//RtmpHRAmUq/downloaded_packages

The downloaded binary packages are in
	/var/folders/q5/3j5mlj012vv_ys8bj7mz3vb00000gn/T//RtmpHRAmUq/downloaded_packages


In [ ]:
### importing the unlabelled CONSTELLATION.csv

## import the data speedily
dataset <- read_csv(file = "~/Downloads/ConstellationCLEAN.csv")
View(dataset)

In [ ]:
## remove the first column which 'might' be a duplicate of row numbers,
## this is worth checking though.
dataset <- dataset[,-1]

## plotting a row
dataset <- data.frame(dataset)
plot(dataset[5,]) ## didn't plot in RStudio :-/ margin issue... -.-

In [ ]:
### importing the labelled datasets already divided into Train and Test
trainLabelled <- read_csv(file = "~/Downloads/Dropbox/See3_train.csv")
testLabelled <- read_csv(file = "~/Downloads/Dropbox/See3_test.csv")

In [ ]:
#change the labels from 1 or 2 into 0 or 1, in prep for XGBoost
trainLabelled$V1[trainLabelled$V1 == 1] <- 0
trainLabelled$V1[trainLabelled$V1 == 2] <- 1

testLabelled$V1[testLabelled$V1 == 1] <- 0
testLabelled$V1[testLabelled$V1 == 2] <- 1

In [ ]:
#randomly assort the rows
set.seed(786)
trainRandIndex <- sample(nrow(trainLabelled))
trainRand <- trainLabelled[trainRandIndex,]

testRandIndex <- sample(nrow(testLabelled))
testRand <- testLabelled[testRandIndex,]

In [ ]:
## XGBoosting this
shot1 <- xgboost(data = as.matrix(trainRand[,2:3198]), label = as.matrix(trainRand[,1]), max.depth = 2, eta = 1, nround = 16, nthread = 2, objective = "binary:logistic", verbose = 1)
shot1Pred <- predict(shot1, as.matrix(testRand[,2:3198]))

shot1PredLabels <- as.numeric(shot1Pred > 0.5)
shot1Error <- mean(as.numeric(shot1PredLabels > 0.5) != testRand[,1])

nlevels(shot1PredLabels)

In [ ]:
## Conc of shot1: XGBoost predicts all observations to be a 0, and predicts nothing to be a 1 :-(
## Time to provide high weightings to penalise every mistaken 1.

trainRandWeights <- factor(as.factor(trainRand$V1), levels = c(0,1), labels = c(1, 100))
trainRandWeights <- as.numeric(as.character(trainRandWeights))

In [ ]:
## XGBoosting Shot2

shot2Matrix <- xgb.DMatrix(data = as.matrix(trainRand[2:3198]), label = as.matrix(trainRand[1]), weight = as.matrix(trainRandWeights))
paras <- list("objective" = "binary:logitraw",
              "max_delta_step" = 10,
              "shot2:eta" = 0.1,
              "eval_metric" = "rmse"
              )
watchlist <- list("shot2" = shot2Matrix)
nround = 128

shot2 <- xgb.train(paras, shot2Matrix, nround, watchlist)
shot2Pred <- predict(shot2, as.matrix(testRand[2:3198]))

shot2PredLabels <- as.numeric(shot2Pred > 0.5)
shot2PredLabels

In [ ]:
## XGBoost still fails to identify even a single 1 in the test set - everything is predicted 0 ;-(
## Doing a PCA visualisation to see where that leads

shot3 <- autoplot(prcomp(trainRand[2:3198]), data = trainRand, color = "V1")

In [ ]:
### -.- was taking too long, switching to python
write_csv( x = trainRand, path = "~/Documents/Python/Const.csv" )